In [ ]:
# Imports libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
#import matplotlib.pyplot as plt
import seaborn as sb
import ipywidgets as widgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
from sklearn import linear_model


In [ ]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [ ]:
# Imports data
data=pd.read_csv('./data.csv')

In [ ]:
table_text = widgets.HTML(value="<p>Here are the top 10 data points from the data set being used. The data set is information recorded from a gym every 10 minutes for a year.</p>")
purpose_text = widgets.HTML(value="<p>Our goal with this data is to use the timestamp and temperature columns then perform multi-linear regression to estimate the amount of people we are likely to find in the gym at a given time and temperature.</p>")
bar_text = widgets.HTML(value="<h4>Total count of individuals through the year at each hour of the day</h4><p>As you can see from the bar graph, people are more likely to visit the gym later in the day.</p>")
scatter_text = widgets.HTML(value="<h4>Number of people in the gym at ranging temperatures</h4><p>At first you would think that the weather effects when people would show up to the gym. However, this scatter plot shows that it does not have much bearing.</p>")
prediction_text = widgets.HTML(value="<h4>Crowdedness Predictor</h4><p>Select a temperature and time and we will predict how many people will be in the gym.</p>")

In [ ]:
line_0 = widgets.HTML(value="<h1>Gym Crowd Prediction</h1><h4>by Jon Rhoads</h4>")
line_1 = widgets.VBox([table_text, purpose_text])
line_2 = widgets.HBox([bar_text])
line_3 = widgets.VBox([scatter_text])
line_4 = widgets.VBox([prediction_text])

In [ ]:
display(line_0)

In [ ]:
display(line_1)

In [ ]:
# Displays raw data
data.head(10)

In [ ]:
display(line_2)

In [ ]:
peopleByHour = []
for i in range(1,25):
    totalPeoplePerHour = 0
    for index, row in data.iterrows():
        if(row['timestamp']/3600 > i - 1 and row['timestamp']/3600 < i):
            totalPeoplePerHour += row['number_people']
    peopleByHour.append(totalPeoplePerHour)

sb.barplot(x=list(range(1,25)), y=peopleByHour)


In [ ]:
display(widgets.HTML(value="<h4>Gym attendees on the weekends</h4><p>In the violin chart we can see that on the weekends there is consistently a lower amount of visitors.</p>"))

In [ ]:
sb.violinplot(y = data['number_people'], x = data['is_weekend'])

In [ ]:
display(line_3)

In [ ]:
sb.catplot(x='number_people', y='temperature', data=data, height=10, aspect=30/10)


In [ ]:
lin_reg = linear_model.LinearRegression()
lin_reg.fit(data[['timestamp','temperature']],data.number_people)

In [ ]:
#lin_reg.coef_

In [ ]:
#lin_reg.intercept_

In [ ]:
display(line_4)

In [ ]:
hours = widgets.Dropdown(
    options=list(range(0,24)),
    continuous_update=True,
    description='Hours:',
)

In [ ]:
minutes = widgets.Dropdown(
    options=list(range(0,61)),
    continuous_update=True,
    description='Minutes:',
)

In [ ]:
temp = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Temp:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
)

In [ ]:
def show_predictions(hour, minute, temp):
    time = (60 * int(minute)) + (3600 * int(hour))
    result = lin_reg.predict([[time, temp]])
    prediction = result[0]
    if prediction < 0:
        prediction = 0
    print("Based on the inputs, you are likely to see {:.0f} people at the gym".format(prediction))
    
interact(show_predictions, hour = hours, minute = minutes, temp = temp)

In [ ]:
!pip freeze > requirements.txt